# Chapitre 4 : Maîtriser les moyennes mobiles en finance

## Objectifs 

- Calculer des moyennes mobiles simples (SMA) et exponentielles (EMA) avec `pandas`
- Visualiser les tendances à l’aide de `matplotlib` et `seaborn`




## 1. Importer les données d'Apple

Les moyennes mobiles sont des indicateurs calculés à partir du prix d’un actif sur une période donnée. Pour les utiliser, il est essentiel de disposer d’un historique de prix suffisamment long. Ici, nous utiliserons les données de l’action Apple (AAPL).


In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

apple = yf.download('AAPL', start='2021-01-01', end='2024-12-31')
# Ça fonctionne même si yfinance renvoie un DataFrame multi-niveaux
apple.columns = apple.columns.droplevel('Ticker')
apple.dropna(inplace=True)
apple.head()


## 2. Calculer une moyenne mobile simple (SMA)

La SMA (Simple Moving Average) est la moyenne arithmétique des prix de clôture sur une période donnée. Elle est utile pour lisser les variations de prix et identifier les tendances de fond.


$$ SMA_t = \frac{1}{n} \sum_{i=0}^{n-1} P_{t-i}
$$

Où :
- $( SMA_t $) est la moyenne mobile simple à la date \( t \)
- $( n $) est la période (ex. 20 jours)
- $( P_{t-i} $) est le prix de clôture à \( t-i \)

In [ ]:
# Option 1: Calculer la moyenne mobile avec cumsum
import numpy as np

def moving_avg(x, n):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[n:] - cumsum[:-n]) / float(n)

# Calcul SMA 20 jours avec NumPy
ma_manual = moving_avg(apple['Close'].values, 20)
apple['SMA_20_manual'] = pd.Series(ma_manual, index=apple.index[-len(ma_manual):])
apple.tail()

In [ ]:
### Option 1 : avec Pandas `rolling()` de pandas
apple['SMA_20'] = apple['Close'].rolling(window=20).mean()
apple[['Close', 'SMA_20']].tail()


In [ ]:
apple.tail()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(apple['Close'], label='Close')
plt.plot(apple['SMA_20'], label='SMA 20')
plt.title('Apple - SMA (20 jours)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Filtrer les 3 derniers mois
data_short = apple.loc[apple.index >= apple.index[-1] - pd.Timedelta(days=90)]

plt.figure(figsize=(12, 6))
plt.plot(data_short['Close'], label='Cours de clôture')
plt.plot(data_short['SMA_20'], label='SMA 20 jours')
plt.title('Apple - SMA 20 jours (période récente)')
plt.xlabel('Date')
plt.ylabel('Prix ($)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import plotly.graph_objects as go

# Préparation des données
plot_data = apple.copy()
plot_data = plot_data.reset_index()

# Créer un graphique vide
fig = go.Figure()

# Ajouter la courbe de clôture
fig.add_trace(go.Scatter(
    x=plot_data['Date'],
    y=plot_data['Close'],
    mode='lines',
    name='Cours de clôture',
    line=dict(color='blue')
))

# Ajouter la moyenne mobile 20 jours
fig.add_trace(go.Scatter(
    x=plot_data['Date'],
    y=plot_data['SMA_20'],
    mode='lines',
    name='Moyenne mobile 20 jours',
    line=dict(color='orange')
))

# Mise en forme
fig.update_layout(
    title='Cours de l’action Apple et moyenne mobile sur 20 jours',
    xaxis_title='Date',
    yaxis_title='Prix ($)',
    template='plotly_white',
    legend_title_text='',
    legend=dict(x=0.01, y=0.99)
)

fig.show()


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Widgets pour les dates
start_date = widgets.DatePicker(description='Date de début', value=pd.to_datetime('2023-01-01'))
end_date = widgets.DatePicker(description='Date de fin', value=pd.to_datetime('2023-06-01'))

# Fonction de traçage
def plot_range(start, end):
    data_range = apple.loc[(apple.index >= pd.to_datetime(start)) & (apple.index <= pd.to_datetime(end))]
    plt.figure(figsize=(12, 6))
    plt.plot(data_range['Close'], label='Cours de clôture')
    plt.plot(data_range['SMA_20'], label='SMA 20 jours')
    plt.title(f"Apple - SMA 20 jours ({start} à {end})")
    plt.xlabel('Date')
    plt.ylabel('Prix ($)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Lier les widgets
ui = widgets.HBox([start_date, end_date])
out = widgets.interactive_output(plot_range, {'start': start_date, 'end': end_date})
display(ui, out)

## 3. Calculer une moyenne mobile exponentielle (EMA)

L’EMA (Exponential Moving Average) donne plus de poids aux prix récents. Elle réagit plus rapidement aux changements de tendance que la SMA.


In [ ]:
apple['EMA_20'] = apple['Close'].ewm(span=20, adjust=False).mean()
apple[['Close', 'EMA_20']].tail()


## 4. Visualiser l’évolution des moyennes mobiles

Visualiser les moyennes mobiles permet d'observer comment elles suivent les fluctuations de prix. Cela aide à interpréter les tendances à court ou long terme.




In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(apple['Close'], label='Close')
plt.plot(apple['SMA_20'], label='SMA 20')
plt.plot(apple['EMA_20'], label='EMA 20')
plt.title('Apple - SMA et EMA (20 jours)')
plt.legend()
plt.grid(True)
plt.show()
